In [1]:
REPLAYS_PATH = '/home/kvu/gfootball-replays/1500/working'

In [3]:
import os
import json
import math

os.listdir(REPLAYS_PATH)

['4333798.json',
 '4582418.json',
 '4458015.json',
 '4373614.json',
 '4452329.json',
 '4940950.json',
 '4804443.json',
 '4793122.json',
 '4571106.json',
 '4693487.json',
 '4773518.json',
 '4550765.json',
 '4857647.json',
 '4219380.json',
 '4239236.json',
 '4329468.json',
 '4611620.json',
 '4340760.json',
 '4546547.json',
 '4703535.json',
 '4940745.json',
 '4417152.json',
 '4452348.json',
 '4667663.json',
 '4657113.json',
 '4708967.json',
 '4607288.json',
 '4713343.json',
 '4379616.json',
 '4665135.json',
 '4607062.json',
 '4299319.json',
 '4324505.json',
 '4435095.json',
 '4942115.json',
 '4803334.json',
 '4497303.json',
 '4899345.json',
 '4899306.json',
 '4737293.json',
 '4462571.json',
 '4245593.json',
 '4628342.json',
 '4761261.json',
 '4714935.json',
 '4815395.json',
 '4935795.json',
 '4510768.json',
 '4271052.json',
 '4813186.json',
 '4885593.json',
 '4914059.json',
 '4636206.json',
 '4243088.json',
 '4207613.json',
 '4737530.json',
 '4492611.json',
 '4268339.json',
 '4807449.json

In [4]:
def get_distance(pos1,pos2):
    return ((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)**0.5

def get_heading(pos1,pos2):
    raw_head=math.atan2(pos1[0]-pos2[0],pos1[1]-pos2[1])/math.pi*180

    if raw_head<0:
        head=360+raw_head
    else:
        head=raw_head
    return head

In [7]:
EntireReplays = []

CENTER_X = 0
CENTER_Y = 0
SIDELINE_X = 0
SIDELINE_Y = 0.42

for game in os.listdir(REPLAYS_PATH):
    GameFullPath = os.path.join(REPLAYS_PATH, game)
    f = open(GameFullPath)
    GameJson = json.load(f)
    for step in GameJson['steps']:
        for i in range(2):
            Obs = step[i]['observation']['players_raw'][0]
            Action = step[i]['action']
            if Obs['ball_owned_player'] != 0:
                continue
            DfRow = {}
            LeftTeamPos = Obs['left_team']
            RightTeamPos = Obs['right_team']
            CurPlayerPos = Obs['left_team'][Obs['active']]

            DfRow['curx'] = get_distance(CurPlayerPos, (CENTER_X, CENTER_Y))
            DfRow['cury'] = get_distance(CurPlayerPos, (SIDELINE_X, SIDELINE_Y))
            
            PlayerNum = 0
            for player in LeftTeamPos:
                PlayerRelativeDist = get_distance(CurPlayerPos, player)
                PlayerHeadingAngle = get_heading(CurPlayerPos, player)
                NameDist = 'l' + str(PlayerNum) + 'd'
                NameAngle = 'l' + str(PlayerNum) + 'a'
                DfRow[NameDist] = PlayerRelativeDist
                DfRow[NameAngle] = PlayerHeadingAngle
                PlayerNum += 1
            
            PlayerNum = 0
            for player in RightTeamPos:
                PlayerRelativeDist = get_distance(CurPlayerPos, player)
                PlayerHeadingAngle = get_heading(CurPlayerPos, player)
                NameDist = 'r' + str(PlayerNum) + 'd'
                NameAngle = 'r' + str(PlayerNum) + 'a'
                DfRow[NameDist] = PlayerRelativeDist
                DfRow[NameAngle] = PlayerHeadingAngle
                PlayerNum += 1
            
            
            DfRow['action'] = Action[0]
            EntireReplays.append(DfRow)
            # print(DfRow)
            # break

In [8]:
len(EntireReplays)

23456

In [9]:
import pandas as pd

df = pd.DataFrame(EntireReplays)

In [10]:
df.head()

,curx,cury,l0d,l0a,l1d,l1a,l2d,l2a,l3d,l3a,...,r6a,r7d,r7a,r8d,r8a,r9d,r9a,r10d,r10a,action
0,0.543576,0.417069,1.289525,73.985375,0.861268,53.304487,0.856903,68.814177,0.785817,74.524472,...,44.144463,0.569356,50.567065,0.464424,89.033096,0.788473,72.978275,0.831361,48.801347,5
1,0.450538,0.867063,0.861713,108.861171,0.000000,0.000000,0.692586,115.625761,0.518578,137.039345,...,209.064484,0.250869,219.976743,0.652120,182.102989,0.647088,120.277901,0.343399,104.964371,14
2,0.554706,0.431475,1.303545,74.162172,0.868621,53.593898,0.865797,69.303670,0.790812,74.676881,...,44.159159,0.573330,50.935798,0.469857,88.979782,0.796278,73.112503,0.838026,49.138577,18
3,0.450538,0.867063,0.857828,108.689457,0.000000,0.000000,0.705833,115.209236,0.529148,135.994014,...,207.923912,0.247224,218.761082,0.651291,181.514736,0.660228,119.922355,0.357504,104.360991,18
4,0.566020,0.445926,1.317611,74.335610,0.876611,53.848444,0.873657,69.798993,0.794901,74.790034,...,44.126110,0.577590,51.270555,0.475337,88.919038,0.803804,73.201115,0.844724,49.471097,18


In [12]:
df.to_csv('replays.csv', sep=',')